In [717]:
#dependencies
import numpy as np
import pandas as pd
import os
import csv

In [835]:
# We Import NBA PLayer's Career Profile
#    This will tell us where they are from

players = './resources/Players.csv'
career= pd.read_csv(players)

# print('Number of Rows', len(career))
career.count()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...,...
3917,3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [749]:
#  We Import NBA Player's Season Profiles
#    This will tell us where they played

player_data = './resources/player_data.csv'
plyr_season = pd.read_csv(player_data)

# Renames the Column 'name' to 'Player'
plyr_season = plyr_season.rename(columns={'name': 'Player'})

# print('Number of Rows', len(plyr_season))
plyr_season.count()

Player        4550
year_start    4550
year_end      4550
position      4549
height        4549
weight        4544
birth_date    4519
college       4248
dtype: int64

In [816]:
# Merge the plyr_season & career Dataframes on 'Players'

Player_Profile_Dirty = pd.merge(plyr_season, career, how='left', on=["Player","Player"])

print('Number of Rows', len(Player_Profile_Dirty))
Player_Profile_Dirty.count()

Number of Rows 4550


Player         4550
year_start     4550
year_end       4550
position       4549
height_x       4549
weight_x       4544
birth_date     4519
college        4248
Unnamed: 0     3814
height_y       3814
weight_y       3814
collage        3497
born           3814
birth_city     3367
birth_state    3353
dtype: int64

In [850]:
# Create DataFrames and edit Column Names


new_columns = [
'Player',
'birth_state',
'position']
PP= Player_Profile_Dirty[new_columns]
# Renames Columns
PPD = PP.rename(columns={
    'birth_state':'State',})
print('length of PPD', len(PPD))



# -----------
htwt = [
    'Player',
    'height_y',
    'weight_x']
player_htwt= Player_Profile_Dirty[htwt]
# Renames Columns
P_htwt = player_htwt.rename(columns={
    'height_y':'Height (cm)',
    'weight_x':'Weight (lbs)'})
print('length of P_htwt', len(P_htwt))



# -----------
college = [
    'Player',
    'college']
player_college= Player_Profile_Dirty[college]
# Renames Columns
P_college = player_college.rename(columns={
   'college':'College'})
print('length of P_college', len(P_college))



# PPD.count()
# P_htwt.count()
# P_college.count()

length of PPD 4550
length of P_htwt 4550
length of P_college 4550


In [851]:
# We Import United States latitude and Longitude

LL= './resources/50_SLat_Lng.csv'
ltlg= pd.read_csv(LL)

# Renames Colums
lat_lng= ltlg.rename(columns={
    'Place Name':'State'})
# lat_lng

In [852]:
#  Merge Player Profile Dataframe  with Lat. & Lng Dataframe

Player_Profile_State = pd.merge(PPD,lat_lng, how='left', on=["State","State"])
# print(len(Player_Profile_State))
Player_Profile_State.count()

Player       4550
State        3353
position     4549
Latitude     2928
Longitude    2928
dtype: int64

In [853]:
# Remove Players not Born in the USA or info not recorded (1622)
# Remove Duplicate Players by -> name and state

Player_State = Player_Profile_State.dropna(axis = 0, how ='any') 


print("Old data frame length:", len(Player_Profile_State),
      "\nNew data frame length:", len(Player_State),
      "\nNumber of Player Not Born In USA: ",(len(Player_Profile_State)-len(Player_State)))
Player_Profile_State.count()
# Player_State.count()

Old data frame length: 4550 
New data frame length: 2927 
Number of Player Not Born In USA:  1623


Player       4550
State        3353
position     4549
Latitude     2928
Longitude    2928
dtype: int64

In [854]:
# ADD USA BORN PLAYERS  College Height and Weight
# Remove Duplicate Players by -> name and state
# Remove player with imcomplete Ht & Wt Data


Player_htwt = pd.merge(Player_State, P_htwt, how='left', on=["Player","Player"])

Player_htwt.drop_duplicates(subset =('Player', 'State'),
                            keep = False, inplace = True)
Player_Profile = Player_htwt.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(Player_State),
      "\nNew data frame length:", len(Player_Profile),
      "\nNumber of Players with Incomplete Ht&Wt Data: ",len(Player_State)-len(Player_Profile))

Player_Profile.count()

Old data frame length: 2927 
New data frame length: 2844 
Number of Players with Incomplete Ht&Wt Data:  83


Player          2844
State           2844
position        2844
Latitude        2844
Longitude       2844
Height (cm)     2844
Weight (lbs)    2844
dtype: int64

In [855]:
# Time to Add Statistics !!!



# We Import NBA Teams Location

tll = './resources/NBATeamLL.csv'
Team_LL= pd.read_csv(tll)
# Team_LL

In [856]:
# # We Import NBA Player Season Stats

Season_stats = './resources/Seasons_Stats.csv'
stats_season = pd.read_csv(Season_stats)
#  Renames Columns

dirty_stats = stats_season.rename(columns={
    'Tm': 'Team',
    'Pos':'Position (Season)',
    'TRB':'Rebounds',
    'STL':'Steals',
    'BLK':'Blocks',
    'AST':'Assists',
    'PTS':'Points',
    'PER':'Player Rating'})
# dirty_stats

In [857]:
season_stats = pd.merge(dirty_stats, Team_LL, how='left', on=["Team","Team"])
# season_stats.count()

In [860]:
# Select only the Columns that are needed.

new_columns = [
    'Year',
    'Player',
    'Position (Season)',
    'Team',
    'Team Name',
    'Team Lat',
    'Team Lng',
    'Rebounds',
    'Steals',
    'Blocks',
    'Assists',
    'Points',
    'Player Rating']
stats_col= season_stats[new_columns]
# stats_col

In [861]:
# Select only the 2010-2015 Seaason statistic

STATS = stats_col.loc[(stats_col['Year']>=2010)&(stats_col['Year']<2015)]
STATS.count()

Year                 2938
Player               2938
Position (Season)    2938
Team                 2938
Team Name            2938
Team Lat             2938
Team Lng             2938
Rebounds             2938
Steals               2938
Blocks               2938
Assists              2938
Points               2938
Player Rating        2936
dtype: int64

In [863]:
# Merge the Clean PLayer Profile with the Clean Player Stats

NBA = pd.merge(Player_Profile,STATS, how='left', on=["Player","Player"])
NBA = NBA.dropna(axis = 0, how ='any')
# NBA.count()

In [864]:
# Add the Players Colleges if they attended

NBAProfile = pd.merge(NBA, P_college, how='left', on=["Player","Player"])
NBAProfile['MapPt'] = pd.Series(3, index=NBAProfile.index)
NBAProfile.count()

Player               2216
State                2216
position             2216
Latitude             2216
Longitude            2216
Height (cm)          2216
Weight (lbs)         2216
Year                 2216
Position (Season)    2216
Team                 2216
Team Name            2216
Team Lat             2216
Team Lng             2216
Rebounds             2216
Steals               2216
Blocks               2216
Assists              2216
Points               2216
Player Rating        2216
College              2044
MapPt                2216
dtype: int64

In [785]:
import gmaps
from api_keys import g_key

In [786]:
gmaps.configure(api_key=g_key)

In [831]:
birthlocations = NBAProfile[["Latitude", "Longitude"]]
Teamlocations = NBAProfile[["Team Lat", "Team Lng"]]
MP = NBAProfile["MapPt"].astype(float)

In [814]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layerB = gmaps.heatmap_layer(birthlocations, weights=(NBAProfile['Player Rating']*NBAProfile['Player Rating']), 
                                 dissipating=False, max_intensity=30000,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layerB)

# Display figure
# print(Player_Profile['State'].value_counts())
fig

Figure(layout=FigureLayout(height='420px'))

In [793]:
# Plot Heatmap
fig2 = gmaps.figure()

# Create heat layer
heat_layerT = gmaps.heatmap_layer(Teamlocations, weights=MP, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=1)
# Add layer
fig2.add_layer(heat_layerT)

# Display figure
# print(Player_Profile['State'].value_counts())
fig2

Figure(layout=FigureLayout(height='420px'))